### This notebook is for the one-time loading of USGS basin polygon locations into the teehr data warehouse
- Use location ID prefix: `usgsbasin-<usgs gage id>`
- NWM 3.0 retrospective RAINRATE (AORC but maybe slightly different?) will be saved as `primary_timeseries`
- No crosswalk data was added at this time

In [ ]:
import geopandas as gpd
import pandas as pd

import teehr
from teehr.evaluation.spark_session_utils import create_spark_session

In [ ]:
BASINS_FILEPATH = "/data/playground/slamont/teehr/warehouse/sedona/usgs_basin_geometry/usgsbasin_geometry_highres.all.parquet"

LOCAL_EV_DIR = "/data/temp_warehouse"

#### Initialize `ev` and point to remote

In [ ]:
spark = create_spark_session()

ev = teehr.Evaluation(
    spark=spark,
    dir_path=LOCAL_EV_DIR,
    create_dir=False
)
ev.set_active_catalog("remote")
ev.active_catalog

#### Locations

In [ ]:
gdf = gpd.read_parquet(BASINS_FILEPATH)  # NOTE: We should include EPSG/CRS in the locations table? (table properties or an extra field?)
gdf.crs

In [ ]:
gdf.head()

In [ ]:
gdf.index.size

In [ ]:
gdf["geometry"] = gdf.geometry.make_valid()

In [ ]:
gdf.to_parquet("/data/playground/slamont/teehr/warehouse/sedona/usgs_basin_geometry/usgsbasin_geometry_highres.all.validated.parquet")

In [ ]:
ev.locations.to_sdf().count()

In [ ]:
ev.locations.to_sdf().show(4)

Append to the locations table in the warehouse

In [ ]:
%%time
ev.locations.load_spatial(
    in_path="/data/playground/slamont/teehr/warehouse/sedona/usgs_basin_geometry/usgsbasin_geometry_highres.all.validated.parquet",
    write_mode="append"
)

In [ ]:
ev.locations.to_sdf().count()

In [ ]:
# %%time  # TODO: Error with geometry handling here
# ev.write.to_warehouse(
#     table_name="locations",
#     source_data=gdf,
#     write_mode="append"
# )